In [37]:
import plotly.express as px
import re
import plotly.io as pio
# pio.renderers.default = "browser"
pio.renderers.default = "notebook_connected"
from collections import OrderedDict
import argparse
from scipy.stats import gmean
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff
import os
import sys
sys.path.append('..')
from util.spec2017_util import (
    bcolors,
    create_folder,
    yaml_load,
    yaml_overwrite,
)
from glob import glob
from scipy.stats import gmean

In [38]:
yml_file = "../util/spec2017_default.yml"
spec2017 = yaml_load(yml_file)
benchname = [dict_["name"] for dict_ in spec2017.values()]
print(benchname)
snapshot_out = "out/"

['perlbench', 'gcc', 'bwaves', 'mcf', 'cactuBSSN', 'namd', 'parest', 'povray', 'lbm', 'omnetpp', 'wrf', 'xalancbmk', 'x264', 'blender', 'cam4', 'deepsjeng', 'imagick', 'leela', 'nab', 'exchange2', 'fotonik3d', 'roms', 'xz']


# rerun from here to add a new bench mark

In [39]:
bench_arr = []
for name in benchname:
    # print(name)
    files = glob(os.path.join(snapshot_out, f"{name}.*"))
    numf = len(files)
    # print(f"spec {name} contains {numf} outputs.")
    if numf == 0: continue
    # 2 comp schemes, 3 xor scheme, 2 constraints, 2 inter intra
    if (numf == 24): bench_arr.append(name)

In [40]:
gem5_dir = "/home/zhewen/repo/gem5-dev/gem5"
num_ss = []
num_sample = 10
bench_arr_anno = []
for name,num in zip(benchname, spec2017.keys()):
    if name in bench_arr:
        ckpt = gem5_dir + f"/ckpt/spec2017/1c-2GB-valgrind/{num}.{name}"
        num_ckpt = len(glob(os.path.join(ckpt, "cpt.*", "")))
        print(f"spec {num}.{name} contains {num_ckpt} ckpts.")
        num_ss.append(num_ckpt*num_sample)
        bench_arr_anno.append(f"{name} ({num_ckpt*num_sample})")
print(num_ss)
bench_arr_full_anno = bench_arr_anno + ["gmean"]
bench_arr_full = bench_arr + ["gmean"]


spec 500.perlbench contains 14 ckpts.
spec 503.bwaves contains 19 ckpts.
[140, 190]


# constrained profiling
Only blocks present in L1 is able to be xored!

In [41]:
xor_schemes = ["none", "rand", "ideal"]
display_name_arr = ["bdi_none", "bdi_rand", "bdi_ideal", 
        "dedup_none", "dedup_rand", "dedup_ideal"]
fig = go.Figure()
fig_profile_num = go.Figure()

bar_set_str_arr = ['_both', '']
for i in range(len(bar_set_str_arr)):
    bar_set_str = bar_set_str_arr[i]
    for comb in display_name_arr:
        print(comb)
        cr_arr = []
        cr_geo_arr = []
        ss_arr = []
        ss_arr_bench = []
        for name,ss,name_anno in zip(bench_arr, num_ss, bench_arr_anno):
            f = open(snapshot_out + f"{name}.{comb}{bar_set_str}", "r")
            str_ = f.readline()
            if len(str_.split(','))==3:
                cr = float(str_.split(',')[0])
                ss = float(str_.split(',')[2])
                cr_geo = float(str_.split(',')[1])
                ss_arr.append(ss)
                ss_arr_bench.append(f"{name_anno}")
                cr_geo_arr.append(cr_geo)
            else: assert False
            cr_arr.append(cr)
            f.close()
        cr_arr.append(gmean(cr_arr))
        cr_geo_arr.append(gmean(cr_geo_arr))
        # print(bench_arr)
        print(cr_arr)
        fig.add_trace(go.Bar(
            x=bench_arr_full,
            y=cr_arr,
            name=f'{comb}',
            opacity=i*0.2+0.8,
        ))
        fig_profile_num.add_trace(go.Bar(
            x=ss_arr_bench,
            y=ss_arr,
            name=f'{comb}',
            opacity=i*0.2+0.8,
        ))

bdi_none
[1.386427864285715, 1.9639288631578926, 1.6501047540556537]
bdi_rand
[2.138521850000001, 3.7944537631578914, 2.8486000564010174]
bdi_ideal
[2.4772564285714296, 3.8059564210526298, 3.0705585828828177]
dedup_none
[1.0410011928571428, 15.361197757894734, 3.9988779913474333]
dedup_rand
[1.7598404285714282, 16.293698042105262, 5.354839731068706]
dedup_ideal
[1.7893535857142866, 16.30018494736842, 5.4006290729302755]
bdi_none
[1.386427864285715, 1.9639288631578926, 1.6501047540556537]
bdi_rand
[1.2031566785714287, 1.9268377473684206, 1.522592428810101]
bdi_ideal
[1.436291235714286, 1.9328156473684208, 1.66615910841873]
dedup_none
[1.0410011928571428, 15.361197757894734, 3.9988779913474333]
dedup_rand
[1.7598404285714282, 16.293698042105262, 5.354839731068706]
dedup_ideal
[1.7893535857142866, 16.30018494736842, 5.4006290729302755]


### arithmetic mean of all snapshots

In [42]:
fig.update_layout(
    title=f"XOR boost snapshot profiling (constrained,arithmetic)\
        <br><sup>BDI and dedup</sup>",
        yaxis_title="compression ratio",
        xaxis_title="combinations",
    font=dict(
        family="Courier New, monospace",
        size=18,
    )
)
fig.update_yaxes(range = [0,3.5])
fig.show()


## plot number of profiles

In [43]:
fig_profile_num.update_layout(
    title=f"XOR boost snapshot profiling\
        <br><sup>BDI and dedup</sup>",
        yaxis_title="# snapshots",
        xaxis_title="combinations",
    font=dict(
        family="Courier New, monospace",
        size=18,
    )
)
fig_profile_num.show()

# unconstrained profiling
any blk in L2 can be xored with another in L2